In [2]:
import numpy as np

class DataPostProcess:
    def __init__(self, config=None):
        self.config = config

    def post_process_vae(self, x_all, y_all_pred, y_all_true):
        x_total = []
        y_total_pred = []
        y_total_true = []

        x_all = x_all.reshape([1,-1])
        y_all_pred = y_all_pred.reshape([1,-1])
        y_all_true = y_all_true.reshape([1,-1])

        ###############################################################################
        # Completed sequence
        ###############################################################################
        for i in range(x_all.shape[-1]):
            x_total.append(x_all[0,i])
        for i in range(y_all_pred.shape[-1]):    
            y_total_pred.append(y_all_pred[0,i])
        for i in range(y_all_true.shape[-1]):
            y_total_true.append(y_all_true[0,i])

        #print(len(x_total))

        del x_all
        del y_all_pred
        del y_all_true

        ###############################################################################
        # Transform in array
        ###############################################################################
        x_total = np.array(x_total).reshape([1,-1])
        y_total_pred = np.array(y_total_pred).reshape([1,-1])
        y_total_true = np.array(y_total_true).reshape([1,-1])

        return x_total, y_total_pred, y_total_true

    def reverse_scale_data(self, input_data, predicted_data, scalar):
        print("input_data shape: ", input_data.shape)
        print("predicted_data shape: ", predicted_data.shape)
        # merge two tensors of shape n * m and n * x to n * (m + x)
        data = np.concatenate((input_data, predicted_data), axis=1)
        # reverse the scaling of the data
        return scalar.inverse_transform(data)
    
    def reverse_scale_data_unetnilm(self, input_data, predicted_data, data_config, scalar_type):
        for idx, app in enumerate(data_config):
            if scalar_type == 'minmax':
                # reverse min max scalar
                input_data[:, idx] = (input_data[:, idx] * (data_config[app]['max'] - data_config[app]['min'])) + data_config[app]['min'] 
                predicted_data[:, idx] = (predicted_data[:, idx] * (data_config[app]['max'] - data_config[app]['min'])) + data_config[app]['min']
            elif scalar_type == 'standard':
                input_data[:, idx] = (input_data[:, idx] * data_config[app]['std']) + data_config[app]['mean']
                predicted_data[:, idx] = (predicted_data[:, idx] * data_config[app]['std']) + data_config[app]['mean']
            
            predicted_data[:, idx] = np.where(predicted_data[:, idx]<0, 0, predicted_data[:, idx])
        

        return input_data, predicted_data